In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, os
pd.set_option('max.columns', 500)

In [ ]:
from backtest.backtest import Backtest, Optimizer
from backtest.data_prep import prepare_data
from backtest.predict_FPPG import predict_FPPG

### Load data

In [ ]:
HISTORY = './data/nba_fantasy_pts.csv'
DF = './data/nba_1_11_19.csv'

In [ ]:
history_df, daily_df = prepare_data(HISTORY, DF)

In [ ]:
daily_df

In [ ]:
history_df.head()

### Make predictions based on predetermined formula

In [ ]:
# Calculate predicted FPPG up until a certain point of time.

my_predictions = predict_FPPG(history_df, '2018-11-05')

In [ ]:
my_predictions.head()

In [ ]:
my_predictions[my_predictions.Nickname == 'Ben Simmons']

In [ ]:
"Mean: {}; Std. Error: {}".format(my_predictions.FPPG_error.mean(), my_predictions.FPPG_error.std())

In [ ]:
my_predictions.FPPG_error.hist(bins=40)

### Optimizer

In [ ]:
# Get optimized lineup for today's picks.

todays_preds = Optimizer(daily_df)
todays_preds.optimize()

### Backtest

In [ ]:
# Setup optimizer for backtest.

my_optimizer = Optimizer(my_predictions)

In [ ]:
# Initiate backtest with optimizer as parameter.

my_backtest = Backtest(my_optimizer)

#### Backtest Dataframe

In [ ]:
my_backtest.create_backtest_df()

In [ ]:
my_backtest.backtest_df

In [ ]:
my_backtest.backtest_results()

In [ ]:
my_backtest.results

In [ ]:
"Mean: {}; Std. Error: {}".format(my_backtest.results.actual_sum.mean(), my_backtest.results.actual_sum.std())

In [ ]:
my_backtest.results.actual_sum.hist(bins=30)

### Conclusion

- Objective: Develop strategies to increase mean total FPPG, and reduce variance.
    - By manipulating predicted FPPG & optimizer.
- More accuracy in FPPG ==> less constraints needed for optimizer, and vice-versa.
    - ie. if perfect FPPG prediction, use determinstic optimizer. Otherwise, use optimizer to hedge against variance.

## Appendix

In [ ]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    
    print(div)
    print("Constraints:")
    
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))